### Graph output of step_04_build_df_iv_skew_csvs process that creates df_iv_final and df_iv_skew csv files.

In [1]:
import pandas as pd
import numpy as np
import pdb
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from barchartacs import db_info
from barchartacs import plotly_utilities as pu
from barchartacs import cme_expirations as cmeexp
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from plotly.subplots import make_subplots

from plotly.graph_objs.layout import Font,Margin
from IPython import display

import datetime
from tqdm import tqdm,tqdm_notebook
import mibian
import py_vollib
import importlib
from py_vollib import black
from py_vollib.black import implied_volatility
import traceback
import db_info
MONTH_CODES = 'FGHJKMNQUVXZ'
DICT_MONTH_NUMS = {MONTH_CODES[i]:i+1 for i in range(len(MONTH_CODES))}

# importlib.reload(db_info)

2021-07-27 08:33:24,425 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


In [2]:
def create_skew_per_date_df(df):
    '''
    Find the first settle_date whose count of rows is equal to max count of rows.
    df: DataFrame from df_iv_final_SS.csv, where SS is like ES, CL, NG, etc.
         df contains data for ONLY ONE SYMBOL
    '''
    # get the first symbol (which should be the only symbol)
    contract = df.symbol.unique()[0]
    # get just that symbol's data
    df12 = df[df.symbol==contract]
    df_counts = df12[['settle_date','moneyness']].groupby('settle_date',as_index=False).count()
    max_count = df_counts.moneyness.max()
    first_max_count_settle_date = df_counts[df_counts.moneyness==max_count].iloc[0].settle_date

    df_ret2 = df12[df12.settle_date==first_max_count_settle_date][['moneyness']]
    all_settle_dates = sorted(df_counts.settle_date.unique())
    for settle_date in all_settle_dates:
        df_temp = df12[df12.settle_date==settle_date][['moneyness','vol_skew']]
        df_ret2 = df_ret2.merge(df_temp,on='moneyness',how='outer')
        df_ret2 = df_ret2.rename(columns={'vol_skew':str(settle_date)})
    df_ret2 = df_ret2.sort_values('moneyness')
    df_ret2.index = list(range(len(df_ret2)))
    df_ret3 = df_ret2.fillna(0)
    df_ret3['csum'] = df_ret3.apply(lambda r: sum([r[c] for c in df_ret2.columns.values]),axis=1)
    df_csum = df_ret3[['moneyness','csum']].groupby('moneyness',as_index=False).max()
    df_ret4 = df_ret3.merge(df_csum,how='inner',on=['moneyness','csum']).drop_duplicates()
    df_ret4.index = list(range(len(df_ret4)))
    
    # eliminate csum column from returned DataFrame
    df_ret4 = df_ret4[[c for c in df_ret4.columns.values if 'csum' not in c]]
    
    # convert zero values to np.NaN, for those Out of the Money columns
    for col in [c for c in df_ret4.columns.values if 'moneyness' not in c]:
        df_ret4[col] = df_ret4[col].apply(lambda v: np.NaN if v==0.0 else v)
    df_ret4[df_ret4.moneyness==0] = 0.0
    return df_ret4


In [3]:
def graph_skew(df,do_plot=False):
    '''
    Graph skew for ONLY ONE symbol.
    If df contains more than one symbol, we will only graph the first symbol in the DataFrames    
    df: DataFrame from df_iv_final_SS.csv, where SS is like ES, CL, NG, etc.
         df contains data for ONLY ONE SYMBOL
    '''
    # get the first symbol (which should be the only symbol)
    contract = df.symbol.unique()[0]

    dfp = create_skew_per_date_df(df[df.symbol==contract])
    
    settle_dates = sorted([c for c in dfp.columns.values if c != 'moneyness'])
    splits = list(np.arange(5,len(settle_dates),5))
    settle_date_groups = np.split(np.array(settle_dates),splits)
    ret_figs = []
    for sdg in settle_date_groups:
        sdg_sorted = [str(c) for c in sorted(sdg)]
        cols = ['moneyness']+list(sdg_sorted)
        dfp_sub = dfp[cols]
        t = f"{contract} {sdg[0]} - {sdg[-1]}"
        f = pu.plotly_plot(dfp_sub,x_column='moneyness',plot_title=t,y_left_label='vol skew')
        ret_figs.append(f)
        if do_plot:
            iplot(f)
    return ret_figs

In [4]:

def graph_skew_subplot_quad(fig_group,rows=1,cols=2):
    '''
    Use subplots to output the results of the method graph_skew above
    '''
    f1 = make_subplots(rows=rows, cols=cols,  
        shared_yaxes=False,
        shared_xaxes=False,                       
        subplot_titles=[fig_group[i]['layout'].title.text for i in range(len(fig_group))],
        horizontal_spacing=0.05,
        vertical_spacing=0.11,                       
        print_grid=False)

    pl_width=450*cols 
    pl_height=400*rows
    title = 'Skew plots<br>'

    f1.update_layout(title=title,                                 
        font= Font(family="Open Sans, sans-serif"),
        showlegend=True,     
        hovermode='x',  
        autosize=True,       
        width=pl_width,       
        height=pl_height,
        plot_bgcolor='#EFECEA', 
        bargap=0.05,
        margin=Margin(
                      l=5,
                      r=5,
                      b=55,
                      t=50
        )
    )    

    for i in range(len(fig_group)):
        x = int(i/2) + 1
        y = i % 2 + 1
        f = fig_group[i]
        l = f.layout
        if y > 1:
            l.yaxis.title=''
        try:
            yaxis = f'yaxis{i+1}'
            xaxis = f'xaxis{i+1}'
            if i < 10:
                yaxis = yaxis.replace('1','') 
                xaxis = xaxis.replace('1','') 
            gname = f'{x,y}'#rfs[i]['layout'].title
            for d in f.data:
                data_y = f'y{i+1}'.replace('1','') 
                d['yaxis']=data_y
                d['legendgroup'] =  gname
                d['name'] = f"{d.name}"
                f1.add_trace(d,x,y)
                f1.update_xaxes(patch=l.xaxis,row=x,col=y)
                f1.update_yaxes(patch=l.yaxis,row=x,col=y)                
        except Exception as e:
            print(f'graph_skew_subplots ERRORS: {str(e)}')
    return f1


def graph_skew_subplots(df,rows=2,cols=2):
    fig_list = graph_skew(df)
    n = rows*cols   
    # using list comprehension 
    fig_groups = [fig_list[i*n:(i + 1)*n] for i in range((len(fig_list) + n - 1) // n )]  
    for fig_group in fig_groups:
        iplot(graph_skew_subplot_quad(fig_group,rows=rows,cols=cols))


In [5]:
sym_to_show = 'ES'

In [6]:
grid_plot=True
# df_ivf2 = df_iv_final[df_iv_final.symbol.str.contains(f'{sym_to_show}')]
df_ivf2 = pd.read_csv(f'./temp_folder/df_iv_final_{sym_to_show}.csv')
df_ivf2.sort_values(['settle_date','symbol','dte','strike'],ascending=False).head()
mcs = {'F':1,'G':2,'H':3,'J':4,'K':5,'M':6,'N':7,'Q':8,'U':9,'V':10,'X':11,'Z':12}
# sorted([f'{s[:2]}{(2000+int(s[-2:]))*100 + int(mcs[s[-3]])}' for s in set(df_iv_final.symbol)])
# clist = [c for c in all_contracts if (c[:2]==f'{SYMBOL_TO_RESEARCH}') & (int(c[-2:])>=19)]
clist = [c for c in df_ivf2.symbol.unique() if (c[:2]==f'{sym_to_show}') & (int(c[-2:])>=20)]
for c in clist:
    dft = df_ivf2[df_ivf2.symbol==c]
    if len(dft)<=0:
        print(f'no data for symbol {c}')
        continue
    if grid_plot:
        graph_skew_subplots(dft)
    else:
        rls = graph_skew(dft,do_plot=True)        

In [7]:
pu.plotly_plot(df_ivf2[df_ivf2.moneyness==0][['settle_date','atm_iv']],x_column='settle_date')

In [8]:
cmeexp.get_expiry('ESH20')

Timestamp('2020-03-20 00:00:00', freq='W-FRI')

### Create charts of Historical Vol Skew vs Atm Vol and Cash Price

In [9]:
pga = db_info.get_db_info('local')

  sec_db


In [10]:
cash_sql = f"select * from sec_schema.underlying_table where symbol='{sym_to_show}Z99';"
df_cash_futures = pga.get_sql(cash_sql)
print(len(df_cash_futures))
df_cash_futures.to_csv(f'./temp_folder/df_cash_futures_{sym_to_show}.csv',index=False)

2741


In [11]:
df_ivf2[['settle_date','atm_iv']].drop_duplicates()

,settle_date,atm_iv
0,20110103,0.197479
14,20110104,0.195791
28,20110105,0.191133
43,20110106,0.192469
57,20110107,0.192759
...,...,...
35602,20210719,0.188529
35612,20210720,0.175984
35623,20210721,0.168984
35633,20210722,0.168871


In [12]:
def plot_skew_vs_atm(df_iv_final,df_iv_skew,df_cash_futures,dist_from_zero=.1,SYMBOL_TO_RESEARCH=sym_to_show):
    # Step 01: create df_skew_2, which holds skew difference between 
    #   positive dist_from_zero and negative dist_from_zero, for each settle_date
    df_skew_2 = df_iv_skew.copy()
    df_skew_2.index.name = None
    skew_range_col = f'iv_skew'
    df_skew_2[skew_range_col] = df_skew_2[dist_from_zero] - df_skew_2[-dist_from_zero]
    df_skew_2.settle_date = df_skew_2.settle_date.astype(int)
    df_skew_2 = df_skew_2[['settle_date',skew_range_col]]
    
    # Step 02: create atm implied vol table, that also has the cash price for each settle_date
    df_atmv = df_iv_final[['settle_date','atm_iv']].drop_duplicates()
    df_cf = df_cash_futures[df_cash_futures.symbol==f'{SYMBOL_TO_RESEARCH}Z99']
    df_atmv = df_atmv.merge(df_cf[['settle_date','close']],how='inner',on='settle_date')
    
    # Step 03: merge skew and atm vol/close tables
    df_ivs = df_skew_2.merge(df_atmv,how='inner',on='settle_date')
    df_ivs = df_ivs.sort_values('settle_date')
    
    # Step 04: plot skew vs atm_iv
    chart_title = f'{SYMBOL_TO_RESEARCH} skew {dist_from_zero*100}% up and down vs atm vol'
    df_ivs_skew_vs_atm_iv = df_ivs[['settle_date',skew_range_col,'atm_iv']]
    pu.iplot(pu.plotly_plot(df_ivs_skew_vs_atm_iv,x_column='settle_date',yaxis2_cols=['atm_iv'],
                      y_left_label='iv_skew',y_right_label='atm_iv',plot_title=chart_title))
    
    # Step 05: plot skew vs close
    chart_title = f'{SYMBOL_TO_RESEARCH} skew {dist_from_zero*100}% up and down vs close'
    df_ivs_skew_vs_close = df_ivs[['settle_date',skew_range_col,'close']]
    pu.iplot(pu.plotly_plot(df_ivs_skew_vs_close,x_column='settle_date',yaxis2_cols=['close'],
                      y_left_label='iv_skew',y_right_label='close',plot_title=chart_title))

    
def plot_atm_vs_close(df_iv_final,df_cash_futures,SYMBOL_TO_RESEARCH=sym_to_show):
    # Step 01: create atm implied vol table, that also has the cash price for each settle_date
    df_atmv = df_iv_final[['settle_date','atm_iv']].drop_duplicates()
    df_cf = df_cash_futures[df_cash_futures.symbol==f'{SYMBOL_TO_RESEARCH}Z99']
    df_atmv = df_atmv.merge(df_cf[['settle_date','close']],how='inner',on='settle_date')

    # Step 02: plot atm_iv vs close
    chart_title = f'{SYMBOL_TO_RESEARCH} atm vol vs close'
    df_atm_vs_close = df_atmv[['settle_date','atm_iv','close']]
    pu.iplot(pu.plotly_plot(df_atm_vs_close,x_column='settle_date',yaxis2_cols=['close'],
                      y_left_label='atm_iv',y_right_label='close',plot_title=chart_title))
    return df_atm_vs_close
dfff = plot_atm_vs_close(df_ivf2,df_cash_futures)

# for d in [.05,.1,.2]:
#     plot_skew_vs_atm(df_iv_final,df_iv_skew,df_cash_futures,d)